In [ ]:
#|default_exp callback.noisy_student

# Noisy student

Callback to apply noisy student self-training (a semi-supervised learning approach) based on: 

Xie, Q., Luong, M. T., Hovy, E., & Le, Q. V. (2020). 
<span style="color:dodgerblue">Self-training with noisy student improves imagenet classification</span>. In Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition (pp. 10687-10698).

In [ ]:
#|export
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

In [ ]:
#|export 
from tsai.imports import *
from tsai.utils import *
from tsai.data.preprocessing import *
from tsai.data.transforms import *
from tsai.models.layers import *
from fastai.callback.all import *

In [ ]:
#|export

# This is an unofficial implementation of noisy student based on:
# Xie, Q., Luong, M. T., Hovy, E., & Le, Q. V. (2020). Self-training with noisy student improves imagenet classification. 
# In Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition (pp. 10687-10698).
# Official tensorflow implementation available in https://github.com/google-research/noisystudent


class NoisyStudent(Callback):
    """A callback to implement the Noisy Student approach. In the original paper this was used in combination with noise: 
        - stochastic depth: .8
        - RandAugment: N=2, M=27
        - dropout: .5
        
    Steps:
        1. Build the dl you will use as a teacher
        2. Create dl2 with the pseudolabels (either soft or hard preds)
        3. Pass any required batch_tfms to the callback
    
    """
    
    def __init__(self, dl2:DataLoader, bs:Optional[int]=None, l2pl_ratio:int=1, batch_tfms:Optional[list]=None, do_setup:bool=True, 
                 pseudolabel_sample_weight:float=1., verbose=False): 
        r'''
        Args:
            dl2:                       dataloader with the pseudolabels
            bs:                        batch size of the new, combined dataloader. If None, it will pick the bs from the labeled dataloader.
            l2pl_ratio:                ratio between labels and pseudolabels in the combined batch
            batch_tfms:                transforms applied to the combined batch. If None, it will pick the batch_tfms from the labeled dataloader (if any)
            do_setup:                  perform a transform setup on the labeled dataset.
            pseudolabel_sample_weight: weight of each pseudolabel sample relative to the labeled one of the loss.
        '''
        
        self.dl2, self.bs, self.l2pl_ratio, self.batch_tfms, self.do_setup, self.verbose = dl2, bs, l2pl_ratio, batch_tfms, do_setup, verbose
        self.pl_sw = pseudolabel_sample_weight
        
    def before_fit(self):
        if self.batch_tfms is None: self.batch_tfms = self.dls.train.after_batch
        self.old_bt = self.dls.train.after_batch # Remove and store dl.train.batch_tfms
        self.old_bs = self.dls.train.bs
        self.dls.train.after_batch = noop        

        if self.do_setup and self.batch_tfms:
            for bt in self.batch_tfms: 
                bt.setup(self.dls.train)

        if self.bs is None: self.bs = self.dls.train.bs
        self.dl2.to(self.dls.device)
        self.dl2.bs = min(len(self.dl2.dataset), int(self.bs / (1 + self.l2pl_ratio)))
        self.dls.train.bs = self.bs - self.dl2.bs
        pv(f'labels / pseudolabels per training batch              : {self.dls.train.bs} / {self.dl2.bs}', self.verbose)
        rel_weight = (self.dls.train.bs/self.dl2.bs) * (len(self.dl2.dataset)/len(self.dls.train.dataset))
        pv(f'relative labeled/ pseudolabel sample weight in dataset: {rel_weight:.1f}', self.verbose)
        
        self.dl2iter = iter(self.dl2)
    
        self.old_loss_func = self.learn.loss_func
        self.learn.loss_func = self.loss
        
    def before_batch(self):
        if self.training:
            X, y = self.x, self.y
            try: X2, y2 = next(self.dl2iter)
            except StopIteration:
                self.dl2iter = iter(self.dl2)
                X2, y2 = next(self.dl2iter)
            if y.ndim == 1 and y2.ndim == 2: y = torch.eye(self.learn.dls.c, device=y.device)[y]
            
            X_comb, y_comb = concat(X, X2), concat(y, y2)
            
            if self.batch_tfms is not None: 
                X_comb = compose_tfms(X_comb, self.batch_tfms, split_idx=0)
                y_comb = compose_tfms(y_comb, self.batch_tfms, split_idx=0)
            self.learn.xb = (X_comb,)
            self.learn.yb = (y_comb,)
            pv(f'\nX: {X.shape}  X2: {X2.shape}  X_comb: {X_comb.shape}', self.verbose)
            pv(f'y: {y.shape}  y2: {y2.shape}  y_comb: {y_comb.shape}', self.verbose)
            
    def loss(self, output, target): 
        if target.ndim == 2: _, target = target.max(dim=1)
        if self.training and self.pl_sw != 1: 
            loss = (1 - self.pl_sw) * self.old_loss_func(output[:self.dls.train.bs], target[:self.dls.train.bs])
            loss += self.pl_sw * self.old_loss_func(output[self.dls.train.bs:], target[self.dls.train.bs:])
            return loss 
        else: 
            return self.old_loss_func(output, target)
    
    def after_fit(self):
        self.dls.train.after_batch = self.old_bt
        self.learn.loss_func = self.old_loss_func
        self.dls.train.bs = self.old_bs
        self.dls.bs = self.old_bs

In [ ]:
from tsai.data.all import *
from tsai.models.all import *
from tsai.tslearner import *

In [ ]:
dsid = 'NATOPS'
X, y, splits = get_UCR_data(dsid, return_split=False)

In [ ]:
pseudolabeled_data = X
soft_preds = True

pseudolabels = ToNumpyCategory()(y) if soft_preds else OneHot()(y)
dsets2 = TSDatasets(pseudolabeled_data, pseudolabels)
dl2 = TSDataLoader(dsets2, num_workers=0)
noisy_student_cb = NoisyStudent(dl2, bs=256, l2pl_ratio=2, verbose=True)
tfms = [None, TSClassification]
learn = TSClassifier(X, y, splits=splits, tfms=tfms, batch_tfms=[TSStandardize(), TSRandomSize(.5)], cbs=noisy_student_cb)
learn.fit_one_cycle(1)

labels / pseudolabels per training batch              : 171 / 85
relative labeled/ pseudolabel sample weight in dataset: 4.0


epoch,train_loss,valid_loss,accuracy,time
0,1.884984,1.809759,0.166667,00:06



X: torch.Size([171, 24, 51])  X2: torch.Size([85, 24, 51])  X_comb: torch.Size([256, 24, 58])
y: torch.Size([171])  y2: torch.Size([85])  y_comb: torch.Size([256])


In [ ]:
pseudolabeled_data = X
soft_preds = False

pseudolabels = ToNumpyCategory()(y) if soft_preds else OneHot()(y)
dsets2 = TSDatasets(pseudolabeled_data, pseudolabels)
dl2 = TSDataLoader(dsets2, num_workers=0)
noisy_student_cb = NoisyStudent(dl2, bs=256, l2pl_ratio=2, verbose=True)
tfms = [None, TSClassification]
learn = TSClassifier(X, y, splits=splits, tfms=tfms, batch_tfms=[TSStandardize(), TSRandomSize(.5)], cbs=noisy_student_cb)
learn.fit_one_cycle(1)

labels / pseudolabels per training batch              : 171 / 85
relative labeled/ pseudolabel sample weight in dataset: 4.0


epoch,train_loss,valid_loss,accuracy,time
0,1.894964,1.814770,0.177778,00:03



X: torch.Size([171, 24, 51])  X2: torch.Size([85, 24, 51])  X_comb: torch.Size([256, 24, 45])
y: torch.Size([171, 6])  y2: torch.Size([85, 6])  y_comb: torch.Size([256, 6])


In [ ]:
#|eval: false
#|hide
from tsai.export import get_nb_name; nb_name = get_nb_name(locals())
from tsai.imports import create_scripts; create_scripts(nb_name)

<IPython.core.display.Javascript object>

/Users/nacho/notebooks/tsai/nbs/026_callback.noisy_student.ipynb saved at 2023-01-21 14:30:23
Correct notebook to script conversion! 😃
Saturday 21/01/23 14:30:25 CET
